## Introduction

This is the third in a series of posts that build a Brainfuck interpreter in Rust.
*   In the [first post](../../../../2024/02/04/brainfuck-interpreter-in-rust-part1), we created the first building blocks for our Brainfuck interpreter in Rust: a data structure for the tape, and a data type for the instructions. The instructions are based on Rust's enums, which allow to build powerful [algebraic data types](https://en.wikipedia.org/wiki/Algebraic_data_type).
*   In the [second post](../../../../2024/03/31/brainfuck-interpreter-in-rust-part2), we built an execution engine that can execute a Brainfuck program which is represented by an [abstract syntax tree](https://en.wikipedia.org/wiki/Abstract_syntax_tree) in the context of a tape. We used the `Read` and `Write` traits to make our execution function generic, such that we could not only read from standard input and write to standard output, but also, e.g., read from byte slices and write to a vector of bytes. This makes unit testing easy.

Currently, we can only execute programs which are given as abstract syntax trees in Rust data structures, such as ***TODO: use a program with < and >. Maybe the Fibonacci one?***
```rust
vec![
    Read,
    Loop(vec![Write, Dec]),
    Write
];
```
In Brainfuck source code, this looks like this: `,[.-].`

In the current post, we will find ways to transform the latter into the former, i.e., to parse Brainfuck source code and generate an abstract syntax tree.

***Please note:** I intentionally did this first without using libraries. I wanted to build a simple parser from scratch, and I found that I learned a lot this way. Therefore, I found it appropriate to write a blog post about it. However, building a parser with the help of libraries that are designed for this purpose is **much** easier and less error-prone. In a future post in this series, we will implement a better parser using the [Rust crate nom](https://docs.rs/nom/latest/nom/).*

<!-- TEASER_END -->

## Importing what we did so far into a Jupyter notebook

I've taken the code from the [Jupyter](https://github.com/freininghaus/freininghaus.github.io/blob/main/posts/2024-02-04-brainfuck-interpreter-in-rust-part1/rust-bf-part1.ipynb) [notebooks](https://github.com/freininghaus/freininghaus.github.io/blob/main/posts/2024-03-31-brainfuck-interpreter-in-rust-part2/rust-bf-part-2.ipynb) that the first posts in this series were based on and copied it to proper Rust files in the directory for the new blog post: ***TODO: provide link to src directory here***

Just like in the last post, we will import these into the Jupyter notebook that is the source of the current blog post:

In [2]:
:dep rust_bf = { package = "rust-bf", path = "." }
    
use rust_bf::{instructions::{Instruction, Instruction::*}, tape::Tape, executor::execute};

A small extension compared to the last post is that the function that executes Brainfuck code now creates an empty Tape, so we can run programs more easily. We'll show this with a somewhat silly example program that reads a number $n$, then reads $n$ bytes, decreases each of them by one, and writes them. Finally, it writes a newline character (`\n`):

In [3]:
let transform_input = vec![
    // read number of items (n)
    Read,
    
    Loop(vec![
        // decrease n by one
        Dec, 

        // go right, read input, decrease by one, write output, go left
        Right, Read, Dec, Write, Left
    ]), 

    // set current cell to 10 (ASCII code for newline, \n)
    Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc,

    // write output
    Write
];

We will now execute this program and see what it does with the byte sequence `b"\x0cIfmmp!Xpsme\""` as input:

In [4]:
let encrypted_message = b"\x0cIfmmp!Xpsme\"";
let mut m: &[u8] = encrypted_message;
execute(&transform_input, &mut m, &mut std::io::stdout());

Hello World!


Note how we can mix and match input and output objects with the generic function `execute` as long as input and output implement the `Read` and `Write` traits, respectively:

In [5]:
// Vec<u8> implements Write. Written bytes will be appended.
let mut output = Vec::new();

let mut m: &[u8] = encrypted_message;
execute(&transform_input, &mut m, &mut output);

println!("Output as bytes: {:?}", output);
println!("Output as str:   {:?}", std::str::from_utf8(&output));

Output as bytes: [72, 101, 108, 108, 111, 32, 87, 111, 114, 108, 100, 33, 10]
Output as str:   Ok("Hello World!\n")


## Step 1: parse programs without loops
Since the hardest part of parsing Brainfuck code is to handle loops, we will start simple and first parse loop-less programs. A simple example is a variant of `transform_input` which does not read the byte count and uses a hard-coded number of input bytes instead. Then we can unroll the loop. For 12 input bytes, which happens to be the length of the string `Hello World!`, this is equivalent to the following source code. Note that all characters which are not mapped to Brainfuck instructions are considered comments: ***TODO: footnote about using bytes to avoid Unicode complications (especially lengths later on and iteration over characters)***

In [6]:
let transform_12_input_bytes_source = 
b"  ,-.         Repeat 12 times:
    ,-.         * read a number
    ,-.         * decrease by one
    ,-.         * write it
    ,-.
    ,-.
    ,-.
    ,-.
    ,-.
    ,-.
    ,-.
    ,-.

    >           go right to an empty cell
    ++++++++++  store 10 (\n)
    .           write output
";

We will split the parsing process into two functions, the first of which takes a single input character and returns an *optional* instruction:

In [7]:
fn parse_simple_instruction(c: &u8) -> Option<Instruction> {
    match c {
        b'<' => Some(Left),
        b'>' => Some(Right),
        b'+' => Some(Inc),
        b'-' => Some(Dec),
        b',' => Some(Read),
        b'.' => Some(Write),
        b'[' | b']' => panic!("parse_simple_instruction cannot handle loops"),
        _ => None
    }
}

[`Option<T>`](https://doc.rust-lang.org/std/option/enum.Option.html) is an enum which has the two variants
* `None`, which corresponds to an empty value,
* `Some(T)`, which holds a value of type `T`.

This concept will look familiar to readers who have used optional values in other languages, like, e.g., [`std::optional`](https://en.cppreference.com/w/cpp/utility/optional) in C++ or [`Maybe`](https://wiki.haskell.org/Maybe) in Haskell. ***TODO footnote Java, null, Optional***

We want to apply this function to every character in the Brainfuck source code. This could be done with a `for` loop, but it is easier to make use of iterators by applying a sequence of functions to transform the source into the result we want. I will not describe the basics of iterators in great detail here (see the [relevant chapter in The Rust Programming Language](https://doc.rust-lang.org/book/ch13-02-iterators.html) or the [standard library documentation](https://doc.rust-lang.org/std/iter/trait.Iterator.html)), but much of what you can do with iterators should be easy to follow for readers who have experience with, e.g., streams in Java, views in C++, list transformations in Haskell, or similar constructs in other languages. Here we parse a small Brainfuck source snippet including comments:

In [8]:
&b".-, comment"
    .iter()
    .map(parse_simple_instruction)
    .collect::<Vec<_>>()

[Some(Write), Some(Dec), Some(Read), None, None, None, None, None, None, None, None]

* `.iter()` creates an iterator that allows to iterate through the input bytes,
* `.map(parse_simple_instruction)` applies the function above to each element,
* `.collect::<Vec<_>>()` collects the transformed elements into a `Vec`. Note that the element type need not be specified because the compiler can deduce that it is `Option<Instruction>`.

We see that parsing the instructions worked. It's just a bit inconvenient that the handling of comments results in each instruction being wrapped in `Some`, and that the `None` values from comments and whitespace are in the result.

We could fix this by
* filtering out the `None` values -- note that we use a *closure* here to call a [member of `Option`](https://doc.rust-lang.org/std/option/enum.Option.html#method.is_some) rather than defining a separate function like `parse_simple_instruction(...)`,
* and then unwrapping the instructions from `Some(...)`. This works similarly to unwrapping valid results from the `Ok` variant of `Result`, which we did in the [previous post in this series](../../../../2024/03/31/brainfuck-interpreter-in-rust-part2):

In [9]:
b".-, comment"
    .iter()
    .map(parse_simple_instruction)
    .filter(|opt| opt.is_some())
    .map(|opt| opt.unwrap())
    .collect::<Vec<_>>()

[Write, Dec, Read]

This works just fine, but the following solution is a bit more elegant:

In [10]:
b".-, comment"
    .iter()
    .flat_map(parse_simple_instruction)
    .collect::<Vec<_>>()

[Write, Dec, Read]

In [11]:
// TODO: just for testing
b".-, comment"
    .iter()
    .map(parse_simple_instruction)
    .flatten()
    .collect::<Vec<_>>()

[Write, Dec, Read]

`.flat_map(...)` is equivalent to `.map(...).flatten()`, where [`flatten()`](https://doc.rust-lang.org/std/iter/trait.Iterator.html#method.flatten) flattens a nested structure of iterators.

We have such a nested structure here because the item type of the mapped iterator is `Option<Instruction>`, and `Option<T>` implements the iterator trait, so it is itself an iterator:
* `None` will not yield any elements,
* `Some(x)` will yield the single element `x` when iterated over.

Now we can build a function that can parse any Brainfuck program that has no loops. Note that we do not have to specify that `.collect()` shall collect the resulting iterator into a `Vec` because the compiler deduces this from the return type of the function: ***TODO: footnote about `&str` and `String` and their correspondence to `&[u8]` and `Vec<u8>`***

In [12]:
fn parse_program_without_loops(input: &[u8]) -> Vec<Instruction> {
    input.iter()
        .flat_map(|c| parse_simple_instruction(c))
        .collect()
}

We will now parse the 12 byte transformation program above and execute it to verify that it works:

In [13]:
let transform_12_input_bytes = parse_program_without_loops(transform_12_input_bytes_source);
let encrypted_message = b"Ifmmp!Xpsme\"";
let mut m: &[u8] = encrypted_message;
execute(&transform_12_input_bytes, &mut m, &mut std::io::stdout());

Hello World!


***TODO: add convenience function that takes a program and input bytes, and returns Vec of output bytes***

## Step 2: parse programs with loops

One interesting aspect of loops is that each instruction that starts a loop, `[`, must have a matching loop end, `]`, and vice versa. Therefore, not every combination of characters forms a valid Brainfuck program, and it would be good if our parser could report some details about the first error in the source code. There are two possible erros:
* There could be an unmatched loop end, i.e., a `]` where no loop was currently active. In this case, we will report the index of the unexpected `]` in the sequence of source characters.
* There could be an unmatched `[`. This error could be fixed by adding an `]` at the end, so we will just report "missing loop end at end of input". We could also try to find out the index of the unmatched `[`, but this would complicate the parsing code. We will see in a future post that this is much easier to do when using the [nom](https://docs.rs/nom/latest/nom/) crate for parsing.

We will use an enum to represent the different error conditions:

In [14]:
#[derive(Debug)]
enum ParseError {
    MissingLoopEndAtEndOfInput,
    UnexpectedLoopEnd(usize)
}

In [27]:
fn parse_loop_body(source: &[u8]) -> Result<(Vec<Instruction>, &[u8]), ParseError> {
    todo!()
}

In [32]:
fn parse_next_instruction(source: &[u8]) -> Result<(Option<Instruction>, &[u8]), ParseError> {
    let Some(c) = source.iter().next() else {
        panic!("parse_next_instruction() should not be called with an empty source.");
    };

    let remaining_source = &source[1..];

    match c {
        b'[' => parse_loop_body(remaining_source).map(|(instructions, source_after_loop)| (Some(Loop(instructions)), source_after_loop)),
        b']' => panic!("loop end should be handled in the caller"),
        c => Ok((parse_simple_instruction(c), remaining_source))
    }
}

In [37]:
parse_next_instruction(b"abc")

Ok((None, [98, 99]))

In [50]:
fn parse_loop_body(source: &[u8]) -> Result<(Vec<Instruction>, &[u8]), ParseError> {
    let mut result: Vec<Instruction> = Vec::new();
    let mut remaining = source;

    while remaining.len() > 0 {
        if remaining[0] == b']' {
            return Ok((result, &remaining[1..]));
        }

        let (opt_instruction, source_after_instruction) = parse_next_instruction(remaining)?;

        if let Some(instruction) = opt_instruction {
            result.push(instruction);
        }

        remaining = source_after_instruction;
    }

    // The expected loop end was not found
    Err(ParseError::MissingLoopEndAtEndOfInput)
}

In [51]:
parse_loop_body(b"").map(|(is, rest)| (is, std::str::from_utf8(rest)))

Err(MissingLoopEndAtEndOfInput)

In [52]:
parse_loop_body(b"+>.abc]foo").map(|(is, rest)| (is, std::str::from_utf8(rest)))

Ok(([Inc, Right, Write], Ok("foo")))

In [54]:
parse_loop_body(b"+[<-].]+").map(|(is, rest)| (is, std::str::from_utf8(rest)))

Ok(([Inc, Loop([Left, Dec]), Write], Ok("+")))

In [57]:
fn parse(source: &[u8]) -> Result<(Vec<Instruction>), ParseError> {
    let mut result: Vec<Instruction> = Vec::new();
    let mut remaining = source;

    while remaining.len() > 0 {
        if remaining[0] == b']' {
            return Err(ParseError::UnexpectedLoopEnd(source.len() - remaining.len()));
        }

        let (opt_instruction, source_after_instruction) = parse_next_instruction(remaining)?;

        if let Some(instruction) = opt_instruction {
            result.push(instruction);
        }

        remaining = source_after_instruction;
    }

    Ok(result)
}

In [59]:
parse(b",.")

Ok([Read, Write])

In [60]:
parse(b"]")

Err(UnexpectedLoopEnd(0))

In [62]:
parse(b"[[[]]]]")

Err(UnexpectedLoopEnd(6))

In [63]:
parse(b"[")

Err(MissingLoopEndAtEndOfInput)

In [76]:
let fib = parse(b"
,>>+<<
[->
  [->>+<<]
  >
  [-<+>>+<]
  >
  [-<+>]
  <<.<
]").unwrap();

In [82]:
let mut output = Vec::new();
{
    let input = [12u8];
    let mut input_slice: &[u8] = &input[..];
    execute(&fib, &mut input_slice, &mut output);
}
output

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144]

In [12]:
fn parse_next_instruction(source: &[u8]) -> Result<(Option<Instruction>, &[u8]), ParseError> {
    match source[0] {
        b']' => panic!("should not happen"),
        b'[' => parse_loop(&source[1..]),
        c => Ok((parse_simple_instruction(c), &source[1..]))
    }
}

Error: cannot find function `parse_loop` in this scope

In [9]:
let x = parse_next_instruction(b",.<");

In [57]:
let b: &[u8] = b"Hello world!";
let c: &[u8; 12] = b"Hello world!";
let d: &[u8] = c;
let message = b"\x0dHello World!\n";
println!("{:?}", message);

[13, 72, 101, 108, 108, 111, 32, 87, 111, 114, 108, 100, 33, 10]


In [58]:
let print_counted = vec![Read, Loop(vec![Dec, Right, Read, Write, Left])];

In [59]:
let mut m: &[u8] = message;
execute(&print_counted, &mut m, &mut std::io::stdout());

Hello World!


In [71]:
let print_counted_decrease = vec![Read, Loop(vec![Dec, Right, Read, Dec, Write, Left]), Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Write];
let encrypted_message = b"\x0cIfmmp!Xpsme\"";
let mut m: &[u8] = encrypted_message;
execute(&print_counted_decrease, &mut m, &mut std::io::stdout());

Hello World!


In [64]:
println!("");

Iello World


In [67]:
let n = b'\n';
let t = b'\t';
println!("{:?} {:?}", n, t);

10 9


In [69]:
let x: [u8; 1] = [9];
println!("{:?}", x);

[9]


In [3]:
// 1. Read a byte value  from  standard input.
// 2. Wr ite  all numbers from this value down to zero to standard output.
let program_countdown = vec![
    Read,
    Loop(vec![
        Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, 
        Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, 
        Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, 
        Write,
        Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, 
        Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, 
        Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec,
        Dec]),
    Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, 
    Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, 
    Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, 
    Write,
    Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, 
    Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, 
    Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec, Dec
];

Given a byte with the value 10 as input, the program produces this output:[<sup id="fnref:variable-scope">6</sup>](#fn:variable-scope)

In [16]:
{
    let ten = vec![10];
    let mut input: &[u8] = &ten;
    let mut output = Vec::new();
    
    execute(&program_countdown, &mut input, &mut output);

    println!("remaining input: {:?}", input);
    println!("output: {:?}", output);

};

remaining input: []
output: [58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48]


In [4]:
let mut t = Tape::new();
t.right();
t.set(42);
t.left();
t.left();
t.set(t.get() + 5);
t

Tape { data: [5, 0, 42], pos: 0 }

Even though we cannot write Brainfuck programs in the usual way, we can construct the abstract syntax tree using the variants of the `Instruction` type directly:

In [5]:
// 1. Read a byte value  from  standard input.
// 2. Wr ite  all numbers from this value down to zero to standard output.
let program_countdown = vec![
    Read,
    Loop(vec![Write, Dec]),
    Write
];

 But how can we execute the program?

## Step 1: programs that do not need I/O

We'll start simple and look at programs which do not use the `Read` and `Write` instructions. Since such a program does not create any output, we can only observe the modifications to the tape.

This is a program that will aggregate the values of neighbouring non-zero cells to one of these cells, and then terminate:[<sup id="fnref:initial-state-all-zeroes">2</sup>](#fn:initial-state-all-zeroes)

In [6]:
let program_add = [
    // Go right until the current cell is zero.
    Loop(vec![Right]),  

    // Go to the second non-zero cell from the right.
    Left,
    Left,

    // As long as the current cell is non-zero:
    // * add the value of the right neighbour cell
    // * move the data pointer one cell to the left
    Loop(vec![
        Right,
        Loop(vec![Dec, Left, Inc, Right]),
        Left,
        Left
    ])
];

So if the initial state of the tape is such that it contains the values 3, 4, and 8 surrounded by zeroes,

<table>
<tbody>
  <tr>
    <td style="border-color:black;border-style:solid;border-width:1px">...</td>
    <td style="border-color:black;border-style:solid;border-width:1px">0</td>
    <td style="border-color:black;border-style:solid;border-width:1px">3</td>
    <td style="border-color:black;border-style:solid;border-width:1px">4</td>
    <td style="border-color:black;border-style:solid;border-width:1px">8</td>
    <td style="border-color:black;border-style:solid;border-width:1px">0</td>
    <td style="border-color:black;border-style:solid;border-width:1px">...</td>
  </tr>
</tbody>
</table>

then we would expect that after runing the program, there is only one non-zero cell remaining, which contains the sum of the values:

<table>
<tbody>
  <tr>
    <td style="border-color:black;border-style:solid;border-width:1px">...</td>
    <td style="border-color:black;border-style:solid;border-width:1px">0</td>
    <td style="border-color:black;border-style:solid;border-width:1px">15</td>
    <td style="border-color:black;border-style:solid;border-width:1px">0</td>
    <td style="border-color:black;border-style:solid;border-width:1px">0</td>
    <td style="border-color:black;border-style:solid;border-width:1px">0</td>
    <td style="border-color:black;border-style:solid;border-width:1px">...</td>
  </tr>
</tbody>
</table>

How can we achieve this?

We will first implement the skeleton of the execution function, which takes two parameters for the time being:
*   `tape` is a mutable reference to the tape (`&mut Tape`). It needs to be mutable because the tape contents will be modified during the execution.
*   `instructions` is a *slice* of instructions (`&[Instruction]`). Essentially, a slice combines a pointer to the start of a range of items, and the length of the range.
    Using a slice, rather than a `Vec`, is more flexible because this allows to call the function not only with a `Vec`, but also with fixed-size arrays and parts of a `Vec` or an array.
    Note that the Rust compiler will convert a reference to a `Vec` automatically to a slice when calling the function.[<sup id="fnref:deref-coercion">3</sup>](#fn:deref-coercion)

The function will then loop over `instructions`.

Let's have a look at what we have so far and just print each instruction before we will see how to evaluate them:

In [7]:
fn execute_v1(tape: &mut Tape, instructions: &[Instruction]) {
    for instruction in instructions {
        println!("execute instruction: {:?}", instruction);
    }
}

let mut t = Tape::new();
execute_v1(&mut t, &program_add);

execute instruction: Loop([Right])
execute instruction: Left
execute instruction: Left
execute instruction: Loop([Right, Loop([Dec, Left, Inc, Right]), Left, Left])


Now we want to do something useful with our instructions.

Enum values, such as our instructions of type `Instruction`, are usually evaluated using pattern matching in Rust. A match expression looks like this:

In [8]:
for instruction in [Left, Right, Inc, Dec, Loop(vec![Dec, Right]), Read, Write] {
    let result = match instruction {
        Left => "left",
        Right => "right",
        Inc => "+",
        Dec => "-",
        Loop(body) => "loop",
        _ => "?"
    };
    println!("instruction: {} ", result);
};

instruction: left 
instruction: right 
instruction: + 
instruction: - 
instruction: loop 
instruction: ? 
instruction: ? 


It contains match arms, which consist of a pattern, and some code that is evaluated if the pattern matches the value. In this simple example, the code is just a string constant for each case.

Now we can think about what code to execute for each instruction:
*   `Left` and `Right` are easy: for these, we just have to call `tape.left()` and `tape.right()`, respectively.
*   For `Inc`, we have to increase the value of the current cell: `tape.set(tape.get() + 1)`
*   Analogously for `Dec`: `tape.set(tape.get() - 1)`

When we encounter a `Loop` instruction, we have to check if the current cell is zero. If that is not the case, we repeatedly execute all instructions in the loop body until the current cell becomes zero:

```rust
while tape.get() != 0 {
    execute(&mut tape, body)
}
```

If we put everything together, we end up with this function that can execute any Brainfuck program without `Read` and `Write` instructions:

In [9]:
fn execute_v2(tape: &mut Tape, instructions: &[Instruction]) {
    for instruction in instructions {
        match instruction {
            Left => tape.left(),
            Right => tape.right(),
            Inc => tape.set(tape.get() + 1),
            Dec => tape.set(tape.get() - 1),
            Loop(body) => {
                while tape.get() != 0 {
                    execute_v2(tape, body)
                }
            }
            _ => panic!("Read and Write are not handled yet!")
        }
    }
}

Note that we must handle all cases in the match expression, or the code will not compile. Here we use the `panic!` macro, which usually aborts the process with the given error message, if the `Read` or `Write` instructions are used.

We can now try to execute our program that sums the numbers on the tape:

In [10]:
let mut t = Tape::new();
t.set(3);
t.right();
t.set(4);
t.right();
t.set(8);
t.left();
t.left();

println!("Initial state: {:?}", t);

execute_v2(&mut t, &program_add);

println!("Final state:   {:?}", t);

Initial state: Tape { data: [3, 4, 8], pos: 0 }
Final state:   Tape { data: [0, 15, 0, 0, 0], pos: 0 }


It works!

Note that every cell which has once been the current cell has got the value zero. This is not significant though because every cell which has not been visited yet has the value zero implicitly.

## Step 2: input and output

So far, we have ignored the instructions `Read` and `Write` (denoted by ',' and '.' in Brainfuck source code).

We can implement match arms for them in the execution function as follows.

For `Read`, we could access standard input with the function [`std::io::stdin()`](https://doc.rust-lang.org/std/io/fn.stdin.html) , and read a byte from it with [`read_exact(...)`](https://doc.rust-lang.org/std/io/struct.Stdin.html#method.read_exact). The following function shows how it works:

In [11]:
use std::io::Read;  // needed to bring read_exact(...) into scope

fn read() -> u8 {
    let mut buf: [u8; 1] = [0];
    std::io::stdin().read_exact(&mut buf).unwrap();
    buf[0]
}

`read_exact(...)` tries to fill the given `u8` slice with bytes from standard input. The reason why we call `unwrap()` on its return value is the following:

In Rust, functions that can fail usually return a value of the type `Result<T, E>`, where `T` is the type that a successful execution of the function would yield, and `E` is an error type. For `read_exact(...)`, which returns no useful information in the successful case, `T` is `()`, the empty type.
`Result` is an enum with two variants:
*   A value of type `Ok(T)` is returned if the function execution was successful.
*   An `Err(E)` signals that an error occurred.

Therefore, it is impossible to forget error handling if the return value is used.
There are several ways to work with results and errors. Here we choose to call `.unwrap()` on the result, which unwraps the value from `Ok`, and panics if the result is actually an `Err`.
Using `unwrap` is useful in two situations:
*   If we are 100% sure that there cannot be an `Err` value in the result because we know that some preconditions are fulfilled which guarantee success.
*   If a panic is acceptable, e.g., because we are just experimenting, and not writing production code.

Note that since we do not need the empty result value of `read_exact(...)` at all in this particular case, we could in principle forget to handle errors. 
The code would compile just fine without calling `.unwrap()`, but the compiler would warn about the lack of error handling (at least outside Jupyter notebooks).

In a future post, we will discuss how to handle all errors properly in our Brainfuck interpreter.

Similarly, we can use [`std::io::stdout()`](https://doc.rust-lang.org/std/io/fn.stdout.html) to access standard output for the `Write` instruction, and write a byte to it with [`write_all(...)`](https://doc.rust-lang.org/std/io/struct.Stdout.html#method.write_all-1):

In [12]:
use std::io::Write;  // needed to bring `write(...)` into scope

fn write(value: u8) {
    std::io::stdout().write(&[value]).unwrap();
}

Similar to what we saw for reading, `write(...)` will attempt to write bytes from a `u8` slice to standard output, and return `Ok(n)` if `n` bytes were written successfully. Again, we simply unwrap the result value and postpone proper error handling to a future blog post.

The function which executes Brainfuck code, including input and output, now looks like this:

In [13]:
fn execute_v3(tape: &mut Tape, instructions: &[Instruction]) {
    for instruction in instructions {
        match instruction {
            Left => tape.left(),
            Right => tape.right(),
            Inc => tape.set(tape.get() + 1),
            Dec => tape.set(tape.get() - 1),
            Loop(body) => {
                while tape.get() != 0 {
                    execute_v3(tape, body)
                }
            }
            Read => {
                let mut buf: [u8; 1] = [0];
                std::io::stdin().read_exact(&mut buf).unwrap();
                tape.set(buf[0])
            }
            Write => {
                std::io::stdout().write(&[tape.get()]).unwrap();
            }
        }
    }
}

To test it, we will try the following program which is the equivalent of 
```rust
print!("#\n");
```
in Rust:

In [14]:
let print_hash = vec![
    // load ASCII value for '#' (35)
    Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc,
    Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc,
    Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc,
    Inc, Inc, Inc, Inc, Inc,

    // write to stdout
    Write,

    // clear the current cell
    Loop(vec![Dec]),

    // load ASCII value for '\n' (10)
    Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc, Inc,

    // write to stdout
    Write
];

In [15]:
let mut t = Tape::new();
execute_v3(&mut t, &print_hash);

#


Input also works, but cannot be demonstrated easily in a Jupyter notebook.

## Making input and output more flexible and testable with the traits `Read` and `Write`
We will now make the function `execute(...)` more generic, such that it does not use standard input and standard output directly. Instead, we will make the function accept parameters which define where data should be read from and written to. We can then just pass `std::io::stdin()` and `std::io::stdout()` to this function to get the same behavior that we had so far.

This approach has some advantages
*   We can simulate input even in a Jupyter notebook.
*   More importantly: we can write unit tests for our execution engine. These tests can provide input data and verify output data.

How does this work?

Here is a generic function that writes a byte to a given destination:

In [16]:
fn write<W: Write>(dest: &mut W, value: u8) {
    let mut buf: [u8; 1] = [value];
    dest.write_all(&mut buf).unwrap();
}

The function now accepts a parameter of the generic type `W`. The type is not completely arbitrary: the angle brackets between function name and parameter list tell that the type must implement the [`Write` trait](https://doc.rust-lang.org/std/io/trait.Write.html). Without this restriction, the compiler would not accept the call to `write_all(...)` because this function is provided by the trait.

Rust traits are a bit like interfaces in, e.g., Java and concepts in C++.[<sup id="fnref:trait-objects">4</sup>](#fn:trait-objects) They describe what conditions a type must fulfil. The `Write` trait describes things that bytes can be written to. So we can call this function with standard output:

In [17]:
write(&mut std::io::stdout(), b'#');
write(&mut std::io::stdout(), b'\n');

#


But we can also use other types which implement `Write`. This includes `Vec<u8>`, a vector of bytes. The written bytes will just be appended to the vector:[<sup id="fnref:type-deduction">5</sup>](#fn:type-deduction)

In [18]:
let mut data = Vec::new();
write(&mut data, b'#');
write(&mut data, b'\n');
println!("data={:?}", data);

// We can also interpret the Vec<u8> as an UTF-8 string.
// Note that from_utf8(...) returns a Result because it
// would fail for input which is not valid UTF-8:
println!("data as str: {:?}", std::str::from_utf8(&data));

data=[35, 10]
data as str: Ok("#\n")


We can make the input that we read from generic in the same way by using the `Read` trait.

Our generic execution function then looks like this:

In [19]:
fn execute_v4<R: Read, W: Write>(tape: &mut Tape, instructions: &[Instruction], input: &mut R, output: &mut W) {
    for instruction in instructions {
        match instruction {
            Left => tape.left(),
            Right => tape.right(),
            Inc => tape.set(tape.get() + 1),
            Dec => tape.set(tape.get() - 1),
            Loop(body) => {
                while tape.get() != 0 {
                    execute_v4(tape, body, input, output)
                }
            }
            Read => {
                let mut buf: [u8; 1] = [0];
                input.read_exact(&mut buf).unwrap();
                tape.set(buf[0])
            }
            Write => {
                output.write(&[tape.get()]).unwrap();
            }
        }
    }
}

To test this execution function, we will use the countdown program that we saw earlier:

So it works as expected 🙂

## Summary and outlook
We have combined the tape data structure and the instruction data type from the last post, and implemented a function that can execute the abstract syntax tree for any Brainfuck program in the context of the tape. Input and output were handled in a generic way using the `Read` and `Write` traits, such that program execution can be tested easily.

In the next post, we will implement a parser that transforms Brainfuck source code to an abstract syntax tree.

---

1.  <span id="fn:import-local-crate">The</span> [documentation](https://github.com/evcxr/evcxr/blob/main/COMMON.md) for the Rust Jupyter kernel describes how to do this (search for "*You can use the local work-in-progress crate like this*"). [&#8617;](#fnref:import-local-crate)

1. <span id="fn:initial-state-all-zeroes">Note</span> that Brainfuck programs usually operate on a tape which has all cells initialized to zero. So strictly speaking, a program that only sums the initial cell values does not make much sense. It is only used to test our intermediate step on the way to a full-featured Brainfuck execution engine here.[&#8617;](#fnref:initial-state-all-zeroes)

1.  <span id="fn:deref-coercion">This</span> feature is called [deref coercion](https://doc.rust-lang.org/book/ch15-02-deref.html#implicit-deref-coercions-with-functions-and-methods). [&#8617;](#fnref:deref-coercion)

1.  <span id="fn:trait-objects">Our</span> generic function `write<W: Write>(...)` is like a template in C++ in the sense that the compiler will generate separate assembly for each type that the function is used with. So our use of traits to define conditions that the type must fulfil corresponds more to concepts in C++ than to interfaces in Java. However, traits can also be used in a more dynamic way, such that only one version of the function exists in assembly and machine code, and dispatch is done dynamically at runtime with virtual function calls. This is more like what interfaces in Java and abstract base classes in C++ are used for.

    Here is an example function, which gets a so-called *trait object* as a parameter:
    ```rust
    fn write_trait_object(w: &mut dyn Write, value: u8) {
        let mut buf: [u8; 1] = [value];
        w.write_all(&mut buf).unwrap();
    }
    ```
    It can be called with standard output as the writer like this:
    ```rust
    write_trait_object(&mut std::io::stdout(), b'#');
    write_trait_object(&mut std::io::stdout(), b'\n');
    ```
    The differences to our earlier function `write<W: Write>(...)` are:
    *   Only one version of the function exists for all types in machine code, so the size of the compiled program might be lower.
    *   Due to dynamic dispatch at runtime, there may be a small performance penalty. Moreover, the compiler cannot perform optimizations for specific types that the function is used with. This could harm performance and increase the size of the compiled program.
  
    It can be tempting to use static dispatch and have the compiler generate optimal code for each type to improve the performance, but adding type annotations to functions and structs also has costs. In particular, it can harm developer productivity if done too excessively. The other day, I read a [very interesting blog post about a refactoring to static dispatch that the author considered a mistake](https://jmmv.dev/2023/08/rust-static-dispatch-failed-experiment.html).[&#8617;](#fnref:trait-objects)

1.  <span id="fn:type-deduction">Note</span> that we do not have to state the type `Vec<u8>` explicitly when we create the Vec with `Vec::new()`. We could do it in two different ways:
    ```rust
    let mut data: Vec<u8> = Vec::new();
    ```
    or
    ```rust
    let mut data = Vec::<u8>::new();
    ```
    
    But the Rust compiler will deduce the type automatically here because we call the `write(...)` method of the `Write` trait on it, which `Vec<T>` only implements for `T` is `u8`.[&#8617;](#fnref:type-deduction)

1.  <span id="fn:variable-scope">I</span> have wrapped the code in this cell in braces (`{...}`) because compilation will fail with this error otherwise:

    `Error: The variable input contains a reference with a non-static lifetime so can't be persisted. You can prevent this error by making sure that the variable goes out of scope - i.e. wrapping the code in {}.`

    You would not have this problem outside a Jupyter notebook, because then `input` would have a well-define life time.

    Another way to fix this issue besides the braces would be to redefine the variable `input`:
    ```rust
    let input = 0;
    ```
    Then the old variable `input` would also go out of scope. In Rust, variable names can be reused in the same scope, which can be confusing at first. I found this feature quite useful though after I got used to it.[&#8617;](#fnref:variable-scope)

In [42]:
let s = "äbc";
s[0]

Error: the type `str` cannot be indexed by `{integer}`

In [48]:
"äbc".char_indices().collect::<Vec<_>>()

[(0, 'ä'), (2, 'b'), (3, 'c')]